# Module 3, Week 2 Homework

Least-squares linear regression

**Before class reading: Mathematics in Geology Sections 6.1.1 - 6.2.7 & 7.3.3, Linear Algebra and Its Applications Section 6.5  **

**Last week we:**
- Learn how to deal with bivariate data (fitting lines, curves).
- Apply line fitting to determine the spreading rate of various ocean ridges

**Our goals for today:**
- Computational efficency of solving linearized equations with loops, vectors, and built-in functions
- Least-squares fitting of the Gutenberg-Richter law global EQ catalog

## Setup

Run this cell as it is to setup your environment.

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
import math
import datetime
import time

### Load the global earthquake catalogs

There are two .csv files to open: `raw_global_catalog_2000_2012.csv` and `declustered_global_catalog_2000_2012.csv`. Both have globally distributed earthquake events of magnitude 4 and higher from 2000 - 2012 in the ANSS (Advanced National Seismic System) Comprehensive Catalog. There is a "raw" catalog of all the events and a declustered catalog which has had the aftershock events filtered out. For each file column 0 is the date and time of the event, column 1 is the occurance day of the event (in # of days since 01/01/2000 the first day of the catalog), column 2 is the magnitude of the event, and columns 3 and 4 are the latitude and longitude location coordinates.

Declare variables for the columns of data you will use further.

### Set up data and fit a line to estimate slope and intercept

As we did in class make an array of a range of magnitude values and compute the log10 of the count of events greater than or equal to each magnitude per year:

`m=np.arange(...)
for i in range(0,len(mag),1):
    N[i]=np.log10(np.count_nonzero(mag >= m[i])/numyr)`
    
Do this for both catalogs (unfiltered and declustered). For the declustered catalog set `min_mag` to the the magnitude of completeness.

Plot the observed relationships between earthquake magnitude and log10N for each of the catalogs.

For both catalogs, use the solution method of your choice (i.e. matrix algebra or `polyfit`) to fit the magnitude and log10(number of events) with a best-fit line.

Why did you choose that method?

_Write your answer here._

Plot the model fit-lines and the data. Make 2 subplots with the same x and y axis limits, add annotation diplaying the A and B-value model parameter estimates.

### Estimate the uncertainties of the models

Now you'll estimate the uncertainties of the models. This can be done by estimating the 95% confidence levels by using the students t-test. The 95% confidence interval is the interval around the model estimates within which we are confident the 'true' answer lies given the amount of scatter in the observations. The following steps outline how this is done using the numpy and scipy packages.

Compute the variance and standard deviation estimates of the scatter in the data:

1. `df=(length_of_data) - (number_of_model_parameters)` compute the degrees of freedom i.e. if you have 3000 data points and 2 model parameters to fit you'll have 2998 degrees of freedom

2. `e=(data)-(model predictions)`  compute the prediction error (residuals)

3. `variance=np.sum(e*e)/df` compute the variance

4. `se_y=np.sqrt(variance)/np.sqrt(length_of_data)`    compute the standard error of the estimate

5. `sdev=np.sqrt(variance)`    compute the standard deviation of the estimate

Calculate 95% confidence bounds:

6. ` intrv=stats.t.interval(0.95,df,loc=np.mean(data), scale=stats.sem(data)`    perform a two-sided students t-distribution test

7. `y95=t*sdev*tmp`  compute the lower 95% confidence bound where 

`tmp = np.sqrt(1/length_of_data+((x-np.mean(x))**2)/np.sum((x-np.mean(x))**2))`

8. `a95 = t*(sdev/np.sqrt(length_of_data))` compute the standard error of intercept 

9. `b95 = t*(sdev/np.sqrt(np.sum((x-np.mean(x))**2)))` compute the standard error of slope


Steps 8 and 9 will be important for incorporating Gutenberg Richter uncertainty in Probabilistic Seismic Hazard Analysis (PSHA).

In [ ]:
# This function computes 95% t-test confidence intervals for the intercept, slope, and model 
# of a least-squares linear regression
def conf_interval_95_a_b_model(x,y):
    """
    inputs: x, y data to be fit by a least squares regression - should have the same dimensions
    
    outputs: the 95% condifence bounds for the intercept, slope, and modelY values 

    """
    # Compute the best fit line
    soln =np.polyfit(x,y,1)
    modelY = np.polyval(soln,x)
    
    #Compute the uncertainty in Gutenberg-Richter Parameters
    df=len(y) - 2                                  #degree of freedom
    e=y-modelY                                     #prediction error
    var=np.sum(e**2)/df                            #variance of estimate
    se_y=np.sqrt(var)/np.sqrt(len(y))              #standard error of the estimate
    sdev=np.sqrt(var)  
    #standard deviation
    #Calculate 95% confidence bounds
    t=stats.t.ppf(1-0.05/2,df)                     #two-sided students t-distribution
    tmp=np.sqrt(1/len(x)+((x-np.mean(x))**2)/np.sum((x-np.mean(x))**2))
    tmp=tmp/max(tmp)
    y95=t*tmp*sdev                                 # 95% confidence bounds of model
    se_b=sdev/np.sqrt(np.sum((x-np.mean(x))**2))   #standard error slope
    se_a=sdev/np.sqrt(len(x))                      #standard error of intercept
    a95=se_a*t                                     # 95% confidence bounds of intercept
    b95=se_b*t                                     # 95% confidence bounds of slope
    
    return a95, b95, y95

Use the above function `conf_interval_95_a_b_model` to compute the 95% condifence intervals on the slope, intercept, and modelY for both the raw and declustered catalogs.

Plot the results again, as 2 subplots with the same x and y axis limits, add the lower and upper 95% confidence bounds to the lines and to the A and B value annotations. Your figure should look something like:

<img src="final_HW_fig2.png">

How does the global declustered B-value compare with the declustered  B-value we estimated for the Bay Area?

_Write your answer here._

What does this difference in B-values tell you about the size of earthquakes that occur on strike-slip of faults (the type of faults in the Bay area) in comparison with earthquakes that occur on the diverse fault types (normal, reverse, and strike slip) in the globally-distributed catalog?

_Write your answer here._

In the global declustered catalog what is the predicted occurance rate of M6+ earthquakes? 

_Write your answer here._